# Demostration of the recommenders

In [1]:
import pandas as pd
import numpy as np
from surprise import dump
from evaluation.Evaluate import Evaluate

import model.CollaborativeFilteringRec as CFR
import model.HybridRecommendationSystem as HR
import model.ContentBasedRec as CBR

movie_df = pd.read_csv('./data/external/movies.csv')

## Preprocessing
* Preprocessed data for both content-based and colloborative filtering systems.
* Updates the precalculated dataframes.

In [2]:
print("Prerocessing for Collaborative Filtering Recommender...")
from data_script.preprocess_collaborative import sveta_user_id, hamid_user_id
# import clustering_collaborative #TODO

print("Prerocessing for Content-Based Recommender...")
from data_script.preprocess_content_based import get_df_ContBaseRec
df_ContBaseRec = get_df_ContBaseRec()

from data_script.clustering_content_based import get_labeledMovies
labeledMovies = get_labeledMovies(df_ContBaseRec, clusters=18)


Creating raw dataframes...
Creating dataframes with Sveta and Hamid ratings...
Creating dataframes with anti-Sveta and anti-Hamid ratings...
Creating new ratings dataframe for collaborative filtering ...
Out of total of  138493  users,  64051  are considered lazy and will be removed.
The final number of users is  74442
Randomly choosing 1000 users...
Adding Sviatlana's and Hamid's ratings... we are lazy users, but still...
Sveta 's user id is 48779
Hamid 's user id is 22566
Adding anti-Sviatlana's and anti-Hamid's ratings... they are lazy users, but still...
anti-Sveta 's user id is 102640
anti-Hamid 's user id is 78103
Now, the final number of users is  1004
Filtering movies...
Out of total of  8860  movies,  8298  are considered rare and will be removed.
The final number of movies is  562
Creating the pivot matrix...
The resulting sparcity of the matrix is: 0.7977768640739533
Preparing the final rating matrix...
Data preprocesssing for collaborative filtering modeling is completed!
P

/home/mumu/Documents/DS/movie_recommendation_ProjectCDS/data_script/preprocess_content_based.py:91: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


NameError: name 'df' is not defined

In [ ]:
# pick a user
ratings = pd.read_csv('./data/processed/final_ratings.csv')
random_user = np.random.choice(ratings.userId.unique()) # picks a random user
sveta_user_id, hamid_user_id # or use sveta and hamid's ids.

## Content-based recommender

In [ ]:
cb_df = CBR.recommendation(sveta_user_id, 20, 0)
cb_df.head(20)

## User-based colloborative filtering recommender

In [ ]:
cf_model = CFR.CollaborativeFilteringRecommender()
cf_model.fit_and_predict()
cf_model.recommend(sveta_user_id, 20).merge(movie_df, on='movieId', how='left')

## Hybrid recommender

In [ ]:
h_df = HR.hybrid_recommendation(sveta_user_id)
h_df.head(20)